In [5]:
ls -lh data/

Invalid switch - "".


# Loading The CSV File

In [1]:
%%time

import pandas as pd
df = pd.read_csv('data/fixed.csv')

Wall time: 13.4 s


In [2]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2323984 entries, 0 to 2323983
Data columns (total 14 columns):
0                                     int64
 14                                   int64
 329                                  int64
 0                                    int64
 362                                  int64
 0.1                                  int64
 0.2                                  int64
 0.3                                  int64
 5                                    int64
 14-16\20170817012709_100.h264.txt    object
 2017-08-17 01:27:09                  object
 01:27:09                             object
 0:00:00                              object
 0.0604166666667                      float64
dtypes: float64(1), int64(9), object(4)
memory usage: 591.7 MB


In [10]:
df.describe()

,0,14,329,0,362,0.1,0.2,0.3,5,0.0604166666667
count,2.323984e+06,2.323984e+06,2.323984e+06,2.323984e+06,2.323984e+06,2.323984e+06,2323984.0,2323984.0,2.323984e+06,2.323984e+06
mean,8.271849e+03,2.762049e+02,1.465740e+02,5.330109e+01,4.196587e+07,1.182565e+02,0.0,0.0,8.043612e+00,5.222413e-01
std,5.161792e+03,1.625547e+02,7.780712e+01,9.297647e+01,3.404631e+07,1.532162e+02,0.0,0.0,1.431355e+01,2.707996e-01
min,0.000000e+00,3.000000e+00,3.000000e+00,0.000000e+00,3.620000e+02,0.000000e+00,0.0,0.0,1.000000e+00,0.000000e+00
25%,3.719000e+03,1.460000e+02,9.400000e+01,1.200000e+01,8.056592e+06,1.700000e+01,0.0,0.0,2.000000e+00,3.541667e-01
50%,8.098000e+03,2.720000e+02,1.310000e+02,2.700000e+01,4.154462e+07,6.000000e+01,0.0,0.0,3.000000e+00,5.333333e-01
75%,1.250700e+04,4.050000e+02,2.030000e+02,5.100000e+01,5.209650e+07,1.650000e+02,0.0,0.0,6.000000e+00,7.270833e-01
max,1.847600e+04,6.370000e+02,3.570000e+02,6.430000e+02,1.265195e+08,1.977000e+03,0.0,0.0,1.150000e+02,9.993056e-01


In [21]:
df.columns = ['frame', 'x', 'y', 'obj', 'size', 'seq', 'tbd1', 'tbd2','tbd3', 'filename', 'start', 'path_time', 'delta_time','tbd4']

In [22]:
df.head()

,frame,x,y,obj,size,seq,tbd1,tbd2,tbd3,filename,start,path_time,delta_time,tbd4
0,0,4,291,1,362,0,0,0,5,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09,01:27:09,0:00:00,0.060417
1,0,629,7,2,362,0,0,0,5,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09,01:27:09,0:00:00,0.060417
2,0,253,4,3,362,0,0,0,5,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09,01:27:09,0:00:00,0.060417
3,0,185,4,4,362,0,0,0,5,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09,01:27:09,0:00:00,0.060417
4,1700,5,102,5,362,0,0,0,1,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09,01:28:17,0:01:08,0.061111


In [2]:
%%time
import pandas as pd
df = pd.read_csv('data/fixed.csv',usecols=[1,2,3,5,9,10])
df.columns = ['x', 'y', 'obj', 'seq', 'filename', 'time']


Wall time: 14.7 s


In [3]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2323984 entries, 0 to 2323983
Data columns (total 6 columns):
x           int64
y           int64
obj         int64
seq         int64
filename    object
time        object
dtypes: int64(4), object(2)
memory usage: 319.1 MB


In [4]:
df['time'] = pd.to_datetime(df['time'])

In [5]:
%%time

df_int = df.select_dtypes(include=['int64'])
converted_int = df_int.apply(pd.to_numeric,downcast='unsigned')
                             
df[converted_int.columns] = converted_int
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2323984 entries, 0 to 2323983
Data columns (total 6 columns):
x           uint16
y           uint16
obj         uint16
seq         uint16
filename    object
time        datetime64[ns]
dtypes: datetime64[ns](1), object(1), uint16(4)
memory usage: 175.0 MB
Wall time: 2.42 s


In [6]:
%%time
df_obj = df.select_dtypes(include = ['object']).copy()
converted_obj = pd.DataFrame()

for col in df_obj.columns:
       num_unique_values = len(df_obj[col].unique())
       num_total_values = len(df_obj[col])
       if num_unique_values / num_total_values < 0.5:
           converted_obj.loc[:,col] = df_obj[col].astype('category')
       else:
           converted_obj.loc[:,col] = df_obj[col]

df[converted_obj.columns] = converted_obj

Wall time: 1.47 s


In [7]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2323984 entries, 0 to 2323983
Data columns (total 10 columns):
frame         uint16
x             uint16
y             uint16
obj           uint16
size          uint32
seq           uint16
filename      category
start         category
path_time     category
delta_time    category
dtypes: category(4), uint16(5), uint32(1)
memory usage: 55.5 MB


In [7]:
df

,x,y,obj,seq,filename,time
0,4,291,1,0,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09
1,629,7,2,0,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09
2,253,4,3,0,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09
3,185,4,4,0,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09
4,5,102,5,0,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09
5,6,102,5,1,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09
6,6,102,5,2,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09
7,7,102,5,3,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09
8,9,102,5,4,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09
9,11,102,5,5,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09


In [8]:
df = df.drop_duplicates()

In [9]:
df

,x,y,obj,seq,filename,time
0,4,291,1,0,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09
1,629,7,2,0,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09
2,253,4,3,0,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09
3,185,4,4,0,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09
4,5,102,5,0,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09
5,6,102,5,1,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09
6,6,102,5,2,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09
7,7,102,5,3,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09
8,9,102,5,4,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09
9,11,102,5,5,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09


In [12]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2323984 entries, 0 to 2323983
Data columns (total 10 columns):
frame         uint16
x             uint16
y             uint16
obj           uint16
size          uint32
seq           uint16
filename      category
start         category
path_time     category
delta_time    category
dtypes: category(4), uint16(5), uint32(1)
memory usage: 73.2 MB


In [10]:
%%time
df.to_pickle('data/paths.pkl.xz')

Wall time: 38.5 s


In [11]:
%%time
df2= pd.read_pickle('data/paths.pkl.xz')
df2.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1387329 entries, 0 to 2323982
Data columns (total 6 columns):
x           1387329 non-null uint16
y           1387329 non-null uint16
obj         1387329 non-null uint16
seq         1387329 non-null uint16
filename    1387329 non-null category
time        1387329 non-null datetime64[ns]
dtypes: category(1), datetime64[ns](1), uint16(4)
memory usage: 34.4 MB
Wall time: 2.18 s
